In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('Data/loan.csv')
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [3]:
df.shape

(39717, 111)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(74), int64(13), object(24)
memory usage: 33.6+ MB


In [5]:
df.dropna(axis=1, how='all', inplace=True)                                                          # Drop columns with missing values
df.drop(df.columns[df.nunique() == 1], axis=1, inplace=True)                                        # Drop columns with only one unique value
df.drop(['id','member_id','url','desc','title'], axis=1, inplace=True)                              # Drop columns with high unique values
df.drop(['next_pymnt_d','mths_since_last_delinq','mths_since_last_record'], axis=1, inplace=True)   # Drop columns with high missing values
df.drop(['pub_rec_bankruptcies','out_prncp'], axis=1, inplace=True)                                 # Drop columns with high unique and missing values

In [6]:
# Drop columns with high correlation
cor_matrix = df.corr().abs()                                                            # Get the correlation matrix
upper = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(np.bool))       # Get the upper triangle of the correlation matrix
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]             # Find columns with a correlation greater than 0.95
df.drop(to_drop, axis=1, inplace=True)                                                  # Drop the columns

In [7]:
# Function to clean the percentage column
def clean_percentages(i):
    i = i.str.replace('%', '')
    i = i.astype('float64')
    return i

# Clean the percentage columns
df['int_rate'] = clean_percentages(df['int_rate'])
df['revol_util'] = clean_percentages(df['revol_util'])

In [8]:
# Function to clean the date column
def clean_date(i):
    i = i.str.replace('-', ' ')
    i = i.str[:4]+'20'+i.str[4:]
    i = pd.to_datetime(i).dt.to_period('M')
    return(i)

# Clean the date columns
df['issue_d'] = clean_date(df['issue_d'])
df['last_pymnt_d'] = clean_date(df['last_pymnt_d'])
df['last_credit_pull_d'] = clean_date(df['last_credit_pull_d'])

In [9]:
# Function to clean 'emp_length' column
def clean_emp(i):
    i = i.str.replace('< 1 year','0')           
    i = i.fillna('0')                           
    i = i.str.replace(' ','')
    i = i.str.replace('years','')
    i = i.str.replace('year','')
    i = i.str.replace('+','')
    i = i.astype('int64')
    return i

# Clean the 'emp_length' column
df['emp_length'] = clean_emp(df['emp_length'])

In [10]:
# Function to clean the 'term' column
def clean_term(i):
    i = i.str.replace(' months','').astype('int64')
    i = i/12
    i = i.astype('int64')
    return i

# Clean the 'term' column
df['term'] = clean_term(df['term'])

In [11]:
df['revol_util'] = df['revol_util'].fillna(df['revol_util'].mean())                     # Fill the NaN with mean value
df['emp_title'].fillna('Other', inplace=True)                                           # Fill the NaN with 'Other'

In [12]:
def clean_date_credit(i):
    i = i.str.replace('-', ' ')
    if i.str[4:].astype('int64') > 50:
        i = i.str[:4]+'19'+i.str[4:]
    else:
        i = i.str[:4]+'20'+i.str[4:]
    i = i.str[:4]+'20'+i.str[4:]
    i = pd.to_datetime(i).dt.to_period('M')
    return(i)

In [13]:
df['earliest_cr_line'] = clean_date_credit(df['earliest_cr_line'])

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().